In [2]:
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
from anndata import AnnData
from scipy.sparse import issparse
from scipy.sparse import csr_array
import anndata as ad
from local_metric_functions import check_crop_exists


In [3]:
adata_exp0 = ad.read_h5ad('C:/Users/mdichgan/Documents/Helmholtz/send_to_Jakob/spatial/counts_CPc_exp0_BA28.h5ad')
adata_Yao = ad.read_h5ad(
    'C:/Users/mdichgan/Documents/Helmholtz/send_to_Jakob/sc/Yao_150kcells_subsample_with_annotations_sparse_subset.h5ad')

In [78]:
adata_Yao.obs["celltype"] = adata_Yao.obs["label"]
adata_exp0.layers["raw"] = adata_exp0.X
adata_Yao.layers["raw"] = adata_Yao.X

# sc.pp.normalize_total(adata_exp0)
# sc.pp.normalize_total(adata_Yao)
adata_exp0.layers["lognorm"] = adata_exp0.X
adata_Yao.layers["lognorm"] = adata_Yao.X

In [ ]:
#adata_sp has no x,y coordinates, temporary random:
adata_exp0.obs["x"] = np.random.uniform(0,200,23282)
adata_exp0.obs["y"] = np.random.uniform(0,100,23282)

In [ ]:
#Testen mit adata_exp0 und adata_Yao
np.random.seed(0)
sample_cells = np.random.choice([True, False], adata_Yao.n_obs, p=[0.4,0.6])
sample_genes = np.random.choice([True,False], adata_Yao.n_vars, p = [0.01,0.99])

In [ ]:
adata_test = adata_Yao[sample_cells,sample_genes]

adata_sp, adata_sc = adata_test, adata_test
bins_x, bins_y = list(range(0,201,40)),list(range(0,101,50))
layer = "lognorm"
key = "celltype"

In [6]:
#Test Datensatz erstellen für relative_expression_similarity_across_genes_local:
np.random.seed(0)
layer = "lognorm"
key = "celltype"

sc_gene_A = np.random.normal(1,0.5,20)
sc_gene_B = np.random.normal(-1,0.5,20)
M = np.array([sc_gene_A, sc_gene_B]).T

counts_sc = np.vstack((M,M))

adata_sc = ad.AnnData(counts_sc)
adata_sc.obs_names = [f"Cell_{i:d}" for i in range(adata_sc.n_obs)]
adata_sc.var_names = [f"Gene_{i:d}" for i in range(adata_sc.n_vars)]


counts_sp = np.vstack((M, np.array([sc_gene_B, sc_gene_A]).T))

adata_sp = ad.AnnData(counts_sp)
adata_sp.obs_names = [f"Cell_{i:d}" for i in range(adata_sp.n_obs)]
adata_sp.var_names = [f"Gene_{i:d}" for i in range(adata_sp.n_vars)]

adata_sc.layers[layer] = adata_sc.X
adata_sp.layers[layer] = adata_sp.X

a, b = np.zeros(20), np.ones(20)

adata_sp.obs[["x", "y"]] = np.vstack((np.array([a,a]).T,np.array([b,a]).T))

adata_sp.obs[key] = [f"celltype_{i:d}" for i in range(adata_sp.n_obs)]
adata_sc.obs[key] = [f"celltype_{i:d}" for i in range(adata_sc.n_obs)]

bins_x, bins_y = [0,1,2], [0,1]

In [100]:
#
np.random.seed(1)
layer = "lognorm"
key = "celltype"
adata_sc = ad.AnnData(np.array([[3,0,0]]))
adata_sc.obs_names = [f"Cell_{i:d}" for i in range(adata_sc.n_obs)]
adata_sc.var_names = [f"Gene_{i:d}" for i in range(adata_sc.n_vars)]

adata_sp = ad.AnnData(np.array([[0,1,2]]))
adata_sp.obs_names = [f"Cell_{i:d}" for i in range(adata_sp.n_obs)]
adata_sp.var_names = [f"Gene_{i:d}" for i in range(adata_sp.n_vars)]

adata_sc.layers[layer] = adata_sc.X
adata_sp.layers[layer] = adata_sp.X

adata_sp.obs[["x", "y"]] = np.array([0,0])

adata_sp.obs[key] = [f"celltype_{i:d}" for i in range(adata_sp.n_obs)]
adata_sc.obs[key] = [f"celltype_{i:d}" for i in range(adata_sc.n_obs)]

In [ ]:
x = np.random.normal(2, 1, 100)
y = np.random.normal(1, 1, 100)
H, xedges, yedges = np.histogram2d(x, y, bins=np.array([3,5]))
H

In [ ]:
import matplotlib.pyplot as plt

In [95]:
x = [0.5,0.5,1.5,1.5,1.5]
y = [2.5,1.5,0.5,0.5,1.5]
H, x_edges, y_edges = np.histogram2d(x,y,bins = [[0,1,2],[0,1,2,3]],range=[[0,2],[0,2]])
H.T
# # np.histogram2d(x,y,bins = [2,3],range=[[0,2],[0,2]])
# X, Y = np.meshgrid(x_edges, y_edges)

# # Erstellen Sie die Heatmap
# plt.imshow(H, origin='lower', extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]], cmap='viridis')
# plt.colorbar(label='Counts')  # Farbleiste für die Counts hinzufügen
# plt.xlabel('X')
# plt.ylabel('Y')
# plt.title('2D Histogram Heatmap')
# plt.grid(True, linestyle='--', alpha=0.6)
# plt.show()


array([[0., 2.],
       [1., 1.],
       [1., 0.]])

In [14]:
def get_bin_edges(A: list[list[int]], bins):
    """ Get bins_x and bins_y (the bin edges) from the range matrix A ([[xmin, xmax], [ymin, ymax]]) and bins as in the np.histogram2d function.

    Parameters
    ----------
    A : np.ndarray
    bins : int or array_like or [int, int] or [array, array]
        The bin specification:
        If int, the number of bins for the two dimensions (nx=ny=bins).
        If array_like, the bin edges for the two dimensions (x_edges=y_edges=bins).
        If [int, int], the number of bins in each dimension (nx, ny = bins).
        If [array, array], the bin edges in each dimension (x_edges, y_edges = bins).
        A combination [int, array] or [array, int], where int is the number of bins and array is the bin edges.

    Returns
    -------
    bins_x : array
    bins_y : array
    """
    A = np.array(A)

    if isinstance(bins, int):
        bins_x = np.linspace(A[0, 0], A[0, 1], bins+1)
        bins_y = np.linspace(A[1, 0], A[1, 1], bins+1)
    elif isinstance(bins, (list,np.ndarray)) and len(bins) != 2:
        bins_x = bins
        bins_y = bins
    elif isinstance(bins, (list, tuple)) and len(bins) == 2 and all(isinstance(b, int) for b in bins):
        bins_x = np.linspace(A[0, 0], A[0, 1], bins[0]+1)
        bins_y = np.linspace(A[1, 0], A[1, 1], bins[1]+1)
    elif isinstance(bins, (list, tuple)) and len(bins) == 2 and all(isinstance(b, (list, np.ndarray)) for b in bins):
        bins_x = np.array(bins[0])
        bins_y = np.array(bins[1])
    elif isinstance(bins, (list, tuple)) and len(bins) == 2 and isinstance(bins[0], int) and isinstance(bins[1], (list, np.ndarray)):
        bins_x = np.linspace(A[0, 0], A[0, 1], bins[0]+1)
        bins_y = np.array(bins[1])
    elif isinstance(bins, (list, tuple)) and len(bins) == 2 and isinstance(bins[1], int) and isinstance(bins[0], (list, np.ndarray)):
        bins_x = np.array(bins[0])
        bins_y = np.linspace(A[1, 0], A[1, 1], bins[1]+1)
    else:
        raise ValueError("Invalid 'bins' parameter format")

    return bins_x, bins_y

In [93]:
get_bin_edges([[0,100],[0,20]], bins = [20,20])

(array([  0.,   5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,  50.,
         55.,  60.,  65.,  70.,  75.,  80.,  85.,  90.,  95., 100.]),
 array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 15., 16., 17., 18., 19., 20.]))

In [106]:
def relative_expression_similarity_across_genes_local(
    adata_sp : AnnData, 
    adata_sc : AnnData,
    x_min : int, 
    x_max : int, 
    y_min : int, 
    y_max : int, 
    image : np.ndarray, 
    bins,
    key : str='celltype', 
    layer : str='lognorm',
    min_total_cells : int = 1,      #welche Werte?
    min_n_cells_per_ct : int = 1
    ):

    """Caculate in each gridfield the relative expression similarity across genes. 
    
    If too few cells are in a gridfield, we assign NaN. Further we only include
    celltypes with enough cells per celltype.

    Parameters
    ----------
    spatial_data : AnnData
        annotated ``AnnData`` object with counts from spatial data
    seq_data : AnnData
        annotated ``AnnData`` object with counts scRNAseq data
    x_min : int, 
    x_max : int, 
    y_min : int, 
    y_max : int, 
    image : np.ndarray, 
    bins : int or array_like or [int, int] or [array, array]
        The bin specification:
        If int, the number of bins for the two dimensions (nx=ny=bins).
        If array_like, the bin edges for the two dimensions (x_edges=y_edges=bins).
        If [int, int], the number of bins in each dimension (nx, ny = bins).
        If [array, array], the bin edges in each dimension (x_edges, y_edges = bins).
        A combination [int, array] or [array, int], where int is the number of bins and array is the bin edges.
    key : str
        name of the column containing the cell type information
    layer: str='lognorm'
    min_total_cells: int, optional
        if less than min_total_cells cells are provided, NaN is assigned. By default 50
    min_n_cells_per_ct: int, optional
        only celltypes with at least min_n_cells_per_ct members are considered. By default 10

    Returns
    -------
    gridfield_metric: array of floats  
    """

    range = check_crop_exists(x_min,x_max,y_min,y_max,image)
    bins_x, bins_y = get_bin_edges(range, bins)

    n_bins_x = len(bins_x) - 1
    n_bins_y = len(bins_y) - 1

    ### SET UP
    # set the .X layer of each of the adatas to be log-normalized counts
    adata_sp.X = adata_sp.layers[layer]
    adata_sc.X = adata_sc.layers[layer]

    # take the intersection of genes in adata_sp and adata_sc, as a list
    intersect_genes = list(set(adata_sp.var_names).intersection(set(adata_sc.var_names)))
    n_intersect_genes = len(intersect_genes)

    # subset adata_sc and adata_sp to only include genes in the intersection of adata_sp and adata_sc 
    adata_sc=adata_sc[:,intersect_genes]
    adata_sp=adata_sp[:,intersect_genes]
            
    # find the intersection of unique celltypes in adata_sc and adata_sp
    intersect_celltypes = list(set(adata_sp.obs["celltype"]).intersection(set(adata_sc.obs["celltype"])))

    adata_sc = adata_sc[adata_sc.obs[key].isin(intersect_celltypes), :]
    adata_sp = adata_sp[adata_sp.obs[key].isin(intersect_celltypes), :]

    sp_X = adata_sp.X.toarray()
    exp_sp = pd.DataFrame(sp_X,columns=intersect_genes)
    exp_sp.index = adata_sp.obs.index

    gridfield_metric = np.zeros((n_bins_y,n_bins_x))

    i, j = 0, 0
    for x_start, x_end in zip(bins_x[:-1], bins_x[1:]):
        i = 0
        for y_start, y_end in zip(bins_y[:-1], bins_y[1:]):    
            #hat adata_sp.obs immer x, y Koord? Sonst mit get_cells_location Funktion
            df = adata_sp.obs[["x", "y"]]
            df = df[
            (df["x"] >= x_start)
            & (df["x"] < x_end)
            & (df["y"] >= y_start)
            & (df["y"] < y_end)
            ]

            if len(df) < min_total_cells:
                gridfield_metric[i,j] = np.nan   
                i += 1
                continue  

            sp_local = exp_sp.loc[df.index,:] 
            sp_local[key] = adata_sp.obs.loc[df.index,:][key]           #wieso nicht vor der for-Schleife?

            n_cells_per_ct = sp_local[key].value_counts()
            eligible_ct = n_cells_per_ct.loc[n_cells_per_ct >= min_n_cells_per_ct].index.tolist()

            sp_local = sp_local.loc[sp_local[key].isin(eligible_ct),:]
            sum_sp_local = np.sum(np.sum(sp_local.iloc[:,:-1]))

            adata_sc_local = adata_sc[adata_sc.obs[key].isin(eligible_ct), :]
            sc_X = adata_sc_local.X.toarray()
            sc_local = pd.DataFrame(sc_X, columns = intersect_genes)
            sc_local.index = adata_sc_local.obs.index
            sc_local[key] = adata_sc_local.obs[key]
            sum_sc_local = np.sum(np.sum(sc_local.iloc[:,:-1]))

            if sum_sp_local != 0:
                mean_celltype_sp_normalized=(sp_local.groupby(key).mean().dropna()/sum_sp_local)*(n_intersect_genes)**2     #why still e.g. Meis2 in df? (dropna)
            else: 
                mean_celltype_sp_normalized=0

            if sum_sc_local != 0:
                mean_celltype_sc_normalized=(sc_local.groupby(key).mean().dropna()/sum_sc_local)*(n_intersect_genes)**2
            else: 
                mean_celltype_sc_normalized=0

            mean_celltype_sp_normalized=mean_celltype_sp_normalized.to_numpy()
            pairwise_diff_sp = mean_celltype_sp_normalized[:,:,np.newaxis] - mean_celltype_sp_normalized[:,np.newaxis,:]

            mean_celltype_sc_normalized=mean_celltype_sc_normalized.to_numpy()
            pairwise_diff_sc = mean_celltype_sc_normalized[:,:,np.newaxis] - mean_celltype_sc_normalized[:,np.newaxis,:]

            delta = np.sum(np.abs(pairwise_diff_sp-pairwise_diff_sc))
            gridfield_metric[i,j]  = 1-delta/(2*np.sum(np.abs(pairwise_diff_sc)))
            i+=1
        j+=1     

    return gridfield_metric

In [107]:
relative_expression_similarity_across_genes_local(adata_sp,adata_sc, 0,1,0,1, image = None, bins=1)

array([[0.16666667]])

In [8]:
# Test for relative_expression_similarity_across_cell_type_clusters
np.random.seed(2)
layer = "lognorm"
key = "celltype"

ct_X = np.random.normal(1,0.5,20)
ct_Y = np.random.normal(-1,0.5,20)
M = np.array([ct_X, ct_Y])

counts_sc = np.vstack((M,M))

adata_sc = ad.AnnData(counts_sc)
adata_sc.obs_names = [f"Cell_{i:d}" for i in range(adata_sc.n_obs)]
adata_sc.var_names = [f"Gene_{i:d}" for i in range(adata_sc.n_vars)]


counts_sp = np.vstack((M, np.array([ct_Y, ct_X])))

adata_sp = ad.AnnData(counts_sp)
adata_sp.obs_names = [f"Cell_{i:d}" for i in range(adata_sp.n_obs)]
adata_sp.var_names = [f"Gene_{i:d}" for i in range(adata_sp.n_vars)]

adata_sc.layers[layer] = adata_sc.X
adata_sp.layers[layer] = adata_sp.X

a, b = np.zeros(20), np.ones(20)

adata_sp.obs[["x", "y"]] = np.array([[0,0,1,1],[0,0,0,0]]).T

adata_sp.obs[key] = ["celltype_X","celltype_Y","celltype_X","celltype_Y"]
adata_sc.obs[key] = ["celltype_X","celltype_Y","celltype_X","celltype_Y"]

bins_x, bins_y = [0,1,2], [0,1]

In [16]:
def relative_expression_similarity_across_cell_type_clusters(
    adata_sp : AnnData, 
    adata_sc : AnnData,
    x_min : int, 
    x_max : int, 
    y_min : int, 
    y_max : int, 
    image : np.ndarray, 
    bins,
    key : str='celltype', 
    layer : str='lognorm',
    min_total_cells : int = 1,      #welche Werte?
    min_n_cells_per_ct : int = 1
    ):

    """Caculate in each gridfield the relative expression similarity across cell type clusters. 
    
    If too few cells are in a gridfield, we assign NaN. Further we only include
    celltypes with enough cells per celltype.

    Parameters
    ----------
    spatial_data : AnnData
        annotated ``AnnData`` object with counts from spatial data
    seq_data : AnnData
        annotated ``AnnData`` object with counts scRNAseq data
    x_min : int, 
    x_max : int, 
    y_min : int, 
    y_max : int, 
    image : np.ndarray, 
    bins : int or array_like or [int, int] or [array, array]
        The bin specification:
        If int, the number of bins for the two dimensions (nx=ny=bins).
        If array_like, the bin edges for the two dimensions (x_edges=y_edges=bins).
        If [int, int], the number of bins in each dimension (nx, ny = bins).
        If [array, array], the bin edges in each dimension (x_edges, y_edges = bins).
        A combination [int, array] or [array, int], where int is the number of bins and array is the bin edges.
    key : str
        name of the column containing the cell type information
    layer: str='lognorm'
    min_total_cells: int, optional
        if less than min_total_cells cells are provided, NaN is assigned. By default 50
    min_n_cells_per_ct: int, optional
        only celltypes with at least min_n_cells_per_ct members are considered. By default 10

    Returns
    -------
    gridfield_metric: array of floats  
    """

    
    range = check_crop_exists(x_min,x_max,y_min,y_max,image)
    bins_x, bins_y = get_bin_edges(range, bins)

    n_bins_x = len(bins_x) - 1
    n_bins_y = len(bins_y) - 1

    ### SET UP
    # set the .X layer of each of the adatas to be log-normalized counts
    adata_sp.X = adata_sp.layers[layer]
    adata_sc.X = adata_sc.layers[layer]

    # take the intersection of genes in adata_sp and adata_sc, as a list
    intersect_genes = list(set(adata_sp.var_names).intersection(set(adata_sc.var_names)))

    # subset adata_sc and adata_sp to only include genes in the intersection of adata_sp and adata_sc 
    adata_sc=adata_sc[:,intersect_genes]
    adata_sp=adata_sp[:,intersect_genes]

    # CPM-Normalization      
    sc.pp.normalize_total(adata_sp,target_sum=1e6)
    sc.pp.normalize_total(adata_sc,target_sum=1e6)
            
    # find the intersection of unique celltypes in adata_sc and adata_sp
    intersect_celltypes = list(set(adata_sp.obs["celltype"]).intersection(set(adata_sc.obs["celltype"])))
    n_intersect_celltypes = len(intersect_celltypes)

    adata_sc = adata_sc[adata_sc.obs[key].isin(intersect_celltypes), :]
    adata_sp = adata_sp[adata_sp.obs[key].isin(intersect_celltypes), :]

    sp_X = adata_sp.X.toarray()
    exp_sp = pd.DataFrame(sp_X,columns=intersect_genes)
    exp_sp.index = adata_sp.obs.index

    gridfield_metric = np.zeros((n_bins_y,n_bins_x))

    i, j = 0, 0
    for x_start, x_end in zip(bins_x[:-1], bins_x[1:]):
        i = 0
        for y_start, y_end in zip(bins_y[:-1], bins_y[1:]):    
            #hat adata_sp.obs immer x, y Koord? Sonst mit get_cells_location Funktion
            df = adata_sp.obs[["x", "y"]]
            df = df[
            (df["x"] >= x_start)
            & (df["x"] < x_end)
            & (df["y"] >= y_start)
            & (df["y"] < y_end)
            ]

            if len(df) < min_total_cells:
                gridfield_metric[n_bins_y-1-i,j] = np.nan   
                i += 1
                continue  

            sp_local = exp_sp.loc[df.index,:] 
            sp_local[key] = adata_sp.obs.loc[df.index,:][key]           #wieso nicht vor der for-Schleife?

            n_cells_per_ct = sp_local[key].value_counts()
            eligible_ct = n_cells_per_ct.loc[n_cells_per_ct >= min_n_cells_per_ct].index.tolist()

            sp_local = sp_local.loc[sp_local[key].isin(eligible_ct),:]
            sum_sp_local = np.sum(np.sum(sp_local.iloc[:,:-1]))

            adata_sc_local = adata_sc[adata_sc.obs[key].isin(eligible_ct), :]
            sc_X = adata_sc_local.X.toarray()
            sc_local = pd.DataFrame(sc_X, columns = intersect_genes)
            sc_local.index = adata_sc_local.obs.index
            sc_local[key] = adata_sc_local.obs[key]
            sum_sc_local = np.sum(np.sum(sc_local.iloc[:,:-1]))

            if sum_sp_local != 0:
                mean_celltype_sp_normalized=(sp_local.groupby(key).mean().dropna())*(n_intersect_celltypes)**2     #why still e.g. Meis2 in df? (dropna)
            else: 
                mean_celltype_sp_normalized=0

            if sum_sc_local != 0:
                mean_celltype_sc_normalized=(sc_local.groupby(key).mean().dropna())*(n_intersect_celltypes)**2
            else: 
                mean_celltype_sc_normalized=0

            mean_celltype_sp_normalized=mean_celltype_sp_normalized.T.to_numpy()
            pairwise_diff_sp = mean_celltype_sp_normalized[:,:,np.newaxis] - mean_celltype_sp_normalized[:,np.newaxis,:]

            mean_celltype_sc_normalized=mean_celltype_sc_normalized.T.to_numpy()
            pairwise_diff_sc = mean_celltype_sc_normalized[:,:,np.newaxis] - mean_celltype_sc_normalized[:,np.newaxis,:]

            delta = np.sum(np.abs(pairwise_diff_sp-pairwise_diff_sc))
            gridfield_metric[n_bins_y-1-i,j]  = 1-delta/(2*np.sum(np.abs(pairwise_diff_sc)))
            i+=1
        j+=1     

    return gridfield_metric

In [17]:
relative_expression_similarity_across_cell_type_clusters(adata_sp,adata_sc,0,2,0,1,None,bins=[2,1])

array([[1., 0.]])